In [ ]:
%%capture
import os.path
import glob
import pandas as pd
import numpy as np
import gspread
from pathlib import Path
from google.colab import auth
from google.colab import drive
from google.auth import default
from datetime import datetime, timedelta
from gspread_dataframe import set_with_dataframe
from google.cloud import bigquery
from time import sleep
from dateutil.relativedelta  import relativedelta
from random import randint

# Montagem do Drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')

# Autorização do Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

!cp /content/drive/MyDrive/Colab_Notebooks/pacotes_modulos/modulos_felipe.py /content
from modulos_felipe import etl_funcs
MES_REF = '2023-05'
BASE_VIGENTE = datetime.strptime(MES_REF, "%Y-%m").month % 3

In [ ]:
# Sellers que receberam meses passados
lista_df = []
MES_ANT = (datetime.strptime(MES_REF, '%Y-%m') - relativedelta(months=1)).strftime('%Y-%m')
MES_ANTx2 = (datetime.strptime(MES_REF, '%Y-%m') - relativedelta(months=2)).strftime('%Y-%m')
for mes in [MES_ANT, MES_ANTx2]:
    for arquivo in glob.glob(f'/content/drive/MyDrive/Arquivos NPS/Enviadas para SoluCX/{mes}/*.xlsx'):
        for base in ['Marketplace', 'PME', 'Netshoes']:
            if base in arquivo:
                df_temp = pd.read_excel(arquivo, usecols=['nome_do_cliente']).rename(columns={'nome_do_cliente': 'seller_id'})
                df_temp['base'] = base
                df_temp['mes'] = mes
                lista_df.append(df_temp)
        
receberam_meses_passados = pd.concat(lista_df).drop_duplicates()
receberam_meses_passados['seller_id'] = receberam_meses_passados['seller_id'].str.lower()

### Divisão da base de hunting

In [ ]:
base_hunting = etl_funcs.abre_planilha('Carteiras - Projeto Agentes Parceiro', 'Sellers ativos')[['Helena Seller Code', 'Status']]
base_hunting = base_hunting[base_hunting['Status'].str.contains('ATIVO')][["Helena Seller Code"]].rename(columns={'Helena Seller Code': 'seller_id'})
base_hunting['seller_id'] = base_hunting['seller_id'].str.strip()

In [ ]:
base_hunting = etl_funcs.abre_planilha('Carteiras - Projeto Agentes Parceiro', 'Sellers ativos')[['Helena Seller Code', 'Status']]
base_hunting = base_hunting[base_hunting['Status'].str.contains('ATIVO')][["Helena Seller Code"]].rename(columns={'Helena Seller Code': 'seller_id'}).drop_duplicates()
base_hunting['seller_id'] = base_hunting['seller_id'].str.strip()
base_hunting_already_divided = etl_funcs.abre_planilha('Montagem Base NPS', 'Agente Parceiro').drop_duplicates()
base_hunting_already_divided['seller_id'] = base_hunting_already_divided['seller_id'].str.strip()
base_hunting_new_divisions = base_hunting.merge(base_hunting_already_divided, how="left", on="seller_id")
etl_funcs.exportar_para_planilha('Montagem Base NPS', 'Agente Parceiro', base_hunting_new_divisions, False)

In [ ]:
def separate_hunting_sellers():
    base_hunting = etl_funcs.abre_planilha('Carteiras - Projeto Agentes Parceiro', 'Sellers ativos')[['Helena Seller Code', 'Status']]
    base_hunting = base_hunting[base_hunting['Status'].str.contains('ATIVO')][["Helena Seller Code"]].rename(columns={'Helena Seller Code': 'seller_id'}).drop_duplicates()
    base_hunting['seller_id'] = base_hunting['seller_id'].str.strip()
    base_hunting_already_divided = etl_funcs.abre_planilha('Montagem Base NPS', 'Agente Parceiro').drop_duplicates()
    base_hunting_already_divided['seller_id'] = base_hunting_already_divided['seller_id'].str.strip()
    base_hunting_new_divisions = base_hunting.merge(base_hunting_already_divided, how="left", on="seller_id")
    etl_funcs.exportar_para_planilha('Montagem Base NPS', 'Agente Parceiro', base_hunting_new_divisions, False)
    return "Favor preencher os novos sellers na mão lá na planilha"

###Bases NPS Magalu - Marketplace e PME

In [ ]:
base_geral_aptos_sellers = pd.read_gbq(f"""SELECT h.SellerID as seller_id,
                                                                    --h.Status,
                                                                    --h.Plataforma,
                                                                    --h.PedidoTeste,
                                                                    --v.pedido_processado,
                                                                    if(lower(h.Plataforma) = "ipdv", "PME", "Marketplace") as val_plataforma
                                                            FROM `maga-bigdata.marketplace_analytics.view_dados_sellers_helena` as h
                                                            left join (SELECT  
                                                                so.seller_id, 
                                                                count(distinct so.uuid)  as pedido_processado 
                                                                FROM maga-bigdata.maestro.order as o
                                                                left join maga-bigdata.maestro.sub_order as so on so.order_uuid = o.uuid
                                                                left join maga-bigdata.maestro.sales_channel as sc on o.sales_channel_id = sc.id
                                                                inner join (SELECT distinct status_id, min(date) as date, sub_order_uuid 
                                                                            FROM maga-bigdata.maestro.event 
                                                                            where duplicated = false
                                                                            group by status_id,sub_order_uuid) as processado 
                                                                        on so.uuid = processado.sub_order_uuid and processado.status_id =2
                                                                where so.seller_id <> 'magazineluiza'  
                                                                and sc.organization_uuid = '4da25f48-4193-45de-b4ed-9b8c93b7f987'
                                                                and date(processado.date, "America/Sao_Paulo") BETWEEN "2023-04-01" AND CURRENT_DATE()
                                                                group by so.seller_id ) as v on v.seller_id = h.SellerID
                                                                where h.Status = "Ativo"
                                                                and h.PedidoTeste is not null
                                                                and NOT(v.pedido_processado<2)""", 'maga-bigdata')

In [ ]:
receberam_meses_passados_magalu = receberam_meses_passados.query("base != 'Netshoes'")
base_v1 = base_geral_aptos_sellers[~base_geral_aptos_sellers['seller_id'].isin(receberam_meses_passados_magalu['seller_id'].tolist())]
print(len(base_v1.index))

In [ ]:
# Excluir sellers na esteira
esteira = etl_funcs.abre_planilha('Montagem Base NPS', 'Esteira (Laranja e Vermelho)')
base_v2 = base_v1[~base_v1['seller_id'].isin(esteira['seller_id'].tolist())]
print(len(base_v2.index))

In [ ]:
# Excluir empresas do grupo
empresas_grupo = etl_funcs.abre_planilha('Montagem Base NPS', 'Sellerid Empresa do Grupo')
base_v3 = base_v2[~base_v2['seller_id'].isin(empresas_grupo['seller_id'].tolist())]
print(len(base_v3.index))

In [ ]:
#Excluir KAs (serão adicionados novamente com os e-mails definidos)
kas = etl_funcs.abre_planilha('Montagem Base NPS', 'KA(mkt)')
base_v4 = base_v3[~base_v3['seller_id'].isin(kas['seller_id'].tolist())]
print(len(base_v4.index))

In [ ]:
# Excluir sellers Agente Parceiro que não são da pesquisa atual
agente_parceiro = etl_funcs.abre_planilha('Montagem Base NPS', 'Agente Parceiro')
agente_parceiro = agente_parceiro[agente_parceiro['grupo'] != str(BASE_VIGENTE)]
base_v5 = base_v4[~base_v4['seller_id'].isin(agente_parceiro['seller_id'].tolist())]
print(len(base_v5.index))

In [ ]:
emails_cadastrados = pd.read_gbq("""WITH acesso_mkt AS (select
                                                            bs.sellerIdOpenApi   AS seller_id,
                                                            lower(trim(ul.Email))             AS email,
                                                                                                
                                                        FROM
                                                            `maga-bigdata.integra.user`   AS ul
                                                            INNER JOIN `maga-bigdata.integra.business`   AS bs ON bs.Id = ul.IdBusinessFk
                                                            WHERE
                                                            ul.IsActive is true and
                                                            if((ul.Email) like '%@magazineluiza%',0,1) = 1 
                                                            and if((ul.Email) like '%@luizalabs%',0,1) = 1 
                                                            and if((ul.Email) like '%@donatelo%',0,1) = 1
                                                            and if((ul.Email) like '%@kabum%',0,1) = 1
                                                            and if((ul.Email) like '%@netshoes%',0,1) = 1
                                                            and if((ul.Email) like '%@zattini%',0,1) = 1
                                                            and if((ul.Email) like '%@estantevirtual%',0,1) = 1
                                                            and if((ul.Email) like '%@epocacosmeticos%',0,1) = 1
                                                        ORDER BY
                                                            seller_id, email
                                    ),

                                    acesso_pme AS (select 
                                                        sa.helena_seller_code as seller_id, 
                                                        lower(trim(u.email)) as email
                                                        from `maga-bigdata.ipdv.seller` as s
                                                        inner join `maga-bigdata.ipdv.seller_attributes` as sa on s.id = sa.id_seller
                                                        inner join `maga-bigdata.ipdv.users` as u on s.id = u.id_seller
                                                        where
                                                        u.active is true --usuário ativos
                                                        and s.active is true --seller ativo
                                                        and sa.helena_seller_code is not null --Apenas sellers que já tem o helena
                                                        and if((u.email) like '%@magazineluiza%',0,1) = 1 
                                                        and if((u.email) like '%@luizalabs%',0,1) = 1 
                                                        and if((u.email) like '%@donatelo%',0,1) = 1
                                                        and if((u.email) like '%@kabum%',0,1) = 1
                                                        and if((u.email) like '%@netshoes%',0,1) = 1
                                                        and if((u.email) like '%@estantevirtual%',0,1) = 1
                                                        and if((u.email) like '%@epocacosmeticos%',0,1) = 1
                                                            )
                                                            select
                                                            *
                                                            from acesso_mkt
                                                            union all (select * from acesso_pme)""", 'maga-bigdata')


In [ ]:
# Cruzando emails com base de sellers aptos
base_envio_v1 = base_v5.merge(emails_cadastrados, on='seller_id', how='inner')
len(base_envio_v1.index)

In [ ]:
# Adicionar e-mails KAs novamente
kas['val_plataforma'] = 'Marketplace'
base_envio_v2 = pd.concat([base_envio_v1, kas])
base_envio_v2['email'] = base_envio_v2['email'].str.strip().str.lower()

In [ ]:
# Excluir e-mails que não querem receber
opt_out_comunicacao = etl_funcs.abre_planilha('Montagem Base NPS', 'Opt Out Comunicação')
opt_out_widget = etl_funcs.abre_planilha('Montagem Base NPS', 'Opt Out Widget')
opt_out = pd.concat([opt_out_comunicacao, opt_out_widget])
opt_out['email'] = opt_out['email'].str.strip().str.lower()
base_envio_v3 = base_envio_v2[~base_envio_v2['email'].isin(opt_out_comunicacao['email'].tolist())]
len(base_envio_v3.index)

In [ ]:
# Cruzar com modalidade de envio
base_modalidade_envio = pd.read_gbq("""
SELECT
    SELLERIDML as seller_id,
    ATIVOMLCOL,
    ATIVOAGML
FROM maga-bigdata.mlpdw.sellers_ativos
""", 'maga-bigdata')

base_coletas1 = base_modalidade_envio.query('ATIVOMLCOL == "SIM"')[['seller_id']]
base_coletas1['modalidade'] = 'Magalu coletas'

base_coletas2 = base_modalidade_envio.query('ATIVOAGML == "SIM"')[['seller_id']]
base_coletas2['modalidade'] = 'Agência magalu'
modalidade = pd.concat([base_coletas1, base_coletas2]).drop_duplicates(subset=['seller_id'])

base_envio_v4 = base_envio_v3.merge(modalidade, how='left')

In [ ]:
# Modalidade PME
base_pme = base_envio_v4.query('val_plataforma=="PME"').drop_duplicates()
base_pme['modalidade'] = base_pme['modalidade'].map({
    'Magalu coletas': 'PME (MLE Coletas)',
    'Agência magalu': 'PME (Agência Magalu)'
}).fillna('PME (MLE Postagem)')
etl_funcs.exportar_para_planilha('base_pesquisa_pme', 'main', base_pme.drop_duplicates(subset=['email']), False)

# Modalidade MARKETPLACE
# Definir quem é Magalu Entregas e quem é Envio Parceiro
base_modalidade_envio_2 = pd.read_gbq("""
select
distinct seller as seller_id
from  maga-bigdata.magaluentregas.shipping
where DATETIME(created_at, 'America/Sao_Paulo') >= '2021-01-28'
""", 'maga-bigdata')

base_mkt = base_envio_v4.query('val_plataforma=="Marketplace"').drop_duplicates()
base_mkt['modalidade'] = base_mkt['modalidade'].mask(
    (base_mkt['modalidade'].isna()) & (base_mkt['seller_id'].isin(base_modalidade_envio_2['seller_id'].tolist())),
    'Magalu entregas'
).fillna('Envio parceiro')
base_mkt['modalidade'] = base_mkt['modalidade'].map({
    'Magalu coletas': 'Marketplace (MLE Coletas)',
    'Agência magalu': 'Marketplace (Agência Magalu)',
    'Magalu entregas': 'Marketplace (MLE Postagem)',
    'Envio parceiro': 'Marketplace (Envio Parceiro)',
})

etl_funcs.exportar_para_planilha('base_pesquisa_mkt', 'main', base_mkt.drop_duplicates(subset=['email']), False)

###Base NPS - Nets


In [ ]:
# Base pedidos (Sellers com mais de 2 pedidos)
base_pedidos = pd.read_gbq(f"""Select distinct
                                                            shp.seller_name as LOJISTA,
                                                            shp.seller_id as seller_id,
                                                            n.Ns_Entregas as Tipo_Envio,
                                                            count(osh.delivery_id) as qtd_pedidos
                                                            FROM `maga-bigdata.netshoes_dw.vw_order_full` os, 
                                                        unnest(os.order_shipping) osh,  unnest(osh.shipping_product) shp, unnest(shipping_status_history) hist
                                                        left join maga-bigdata.marketplace_analytics.sellers_nets_nse as n on n.ID_SELLER = shp.seller_id
                                                        left join (SELECT
                                                                        --id,
                                                                        seller_name,
                                                                        default_handling_time,
                                                                        name
                                                                    FROM
                                                                        maga-bigdata.maga_onboarding.distribution_center dc
                                                                    JOIN
                                                                        maga-bigdata.maga_onboarding.handling_time ht ON ht.id = dc.handling_time_id
                                                                    JOIN
                                                                        maga-bigdata.maga_onboarding.origin_address oa ON oa.id = dc.address_id
                                                                    JOIN
                                                                        maga-bigdata.maga_onboarding.distribution_center_organizations dco ON dco.distributioncenter_id = dc.id
                                                                    JOIN
                                                                        maga-bigdata.maga_onboarding.organization o ON o.id = dco.organization_id
                                                                        where name='netshoes'
                                                                        group by seller_name,default_handling_time, name) as d on d.seller_name = shp.seller_id

                                                        left join (SELECT DISTINCT
                                                                            A.PEDS_COD COD_PEDIDO,
                                                                            B.TROD_STR_STA Status_checkin,
                                                                            FROM `maga-bigdata.abacos_nets.aba_tcom_pedsai` A
                                                                            INNER JOIN `maga-bigdata.abacos_nets.aba_ttrd_trodev` B ON A.CLIF_COD = B.CLIF_COD
                                                                            WHERE B.TROD_STR_STA in ('Finalizado', 'Aberto')	
                                                                            group by A.PEDS_COD, B.TROD_STR_STA) as ck on cast(ck.COD_PEDIDO as string) = osh.delivery_id             
                                                                where
                                                                date(os.sale_date) BETWEEN "2023-04-01" AND CURRENT_DATE()
                                                                and os.code not like '%T%'
                                                                and shp.source = '3P' 
                                                                and os.is_ignore_pickup = false
                                                                and NOT(os.amount >= 50000 AND osh.status = 12) 
                                                                and IFNULL(cancel_code,0) <> 100 
                                                                and osh.shipping_status_name in ('CANCELADO', 'DESPACHADO', 'ENTREGUE', 'FATURADO', 'PRONTO PARA FATURAR')
                                                                and ck.Status_checkin is null
                                                                group by 
                                                                1,2,3""", 'maga-bigdata')

base_pedidos['qtd_pedidos']=base_pedidos['qtd_pedidos'].astype(float).astype(int)
base_pedidos['LOJISTA']=base_pedidos['LOJISTA'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.replace(' ','')
base_pedidos_final = base_pedidos.query('qtd_pedidos>2')

# Contatos seller (dados sellers)
dados_contatos_sellers_nets = pd.read_excel(f'/content/base_acessos_nets {MES_REF}.xlsx', usecols=['CNPJ associado', 'E-mail usuário'])
dados_contatos_sellers_nets.rename(columns={'CNPJ associado':'CNPJ',
                                            'E-mail usuário':'email'}, inplace=True)
dados_contatos_sellers_nets['CNPJ'] = dados_contatos_sellers_nets['CNPJ'].astype(float)
#Verificar se tem e-mails diferentes no seller_ID das lojas do grupo.
dados_contatos_sellers_nets['val_email'] = dados_contatos_sellers_nets['email'].str.contains('@magazineluiza|@luizalabs|@donatelo|@kabum|@netshoes|@estantevirtual|@epocacosmeticos')
dados_contatos_sellers_nets_v2 = dados_contatos_sellers_nets.query('val_email==False')

# base com seller_id
base_bq = pd.read_gbq("""SELECT DISTINCT registrationNumber as CNPJ,
                                                                companyName,
                                                                tradeName,
                                                                sellerID as seller_id,
                                                                name as LOJISTA,
                                                                active,
                                                                fastLane,
                                                                
                                                FROM
                                                (SELECT supplier.registrationNumber,
                                                        supplier.companyName,
                                                        supplier.tradeName,
                                                        seller.element._id AS sellerID,
                                                        seller.element.name,
                                                        seller.element.active,
                                                        seller.element.fastLane,
                                                        seller.element.createdDate,
                                                        CASE
                                                            WHEN address.element.type = 'DISTRIBUTION_CENTER' THEN address.element.zipcode
                                                            ELSE NULL
                                                        END AS distributionCenterZipcode,
                                                        CASE
                                                            WHEN address.element.type = 'DISTRIBUTION_CENTER' THEN address.element.street
                                                            ELSE NULL
                                                        END AS distributionCenterStreet,
                                                        CASE
                                                            WHEN address.element.type = 'DISTRIBUTION_CENTER' THEN address.element.number
                                                            ELSE NULL
                                                        END AS distributionCenterNumber,
                                                        CASE
                                                            WHEN address.element.type = 'DISTRIBUTION_CENTER' THEN address.element.neighborhood
                                                            ELSE NULL
                                                        END AS distributionCenterNeighborhood,
                                                        CASE
                                                            WHEN address.element.type = 'DISTRIBUTION_CENTER' THEN address.element.city
                                                            ELSE NULL
                                                        END AS distributionCenterCity,
                                                        CASE
                                                            WHEN address.element.type = 'DISTRIBUTION_CENTER' THEN address.element.state
                                                            ELSE NULL
                                                        END AS distributionCenterState,
                                                        NULL AS returnZipcode,
                                                        NULL AS returnStreet,
                                                        NULL AS returnNumber,
                                                        NULL AS returnNeighborhood,
                                                        NULL AS returnCity,
                                                        NULL AS returnState
                                                FROM netshoes_marketplace.suppliers supplier,
                                                        UNNEST(supplier.sellers.list) seller,
                                                        UNNEST(seller.element.addresses.list) address
                                                UNION ALL SELECT supplier.registrationNumber,
                                                                    supplier.companyName,
                                                                    supplier.tradeName,
                                                                    seller.element._id AS sellerID,
                                                                    seller.element.name,
                                                                    seller.element.active,
                                                                    seller.element.fastLane,
                                                                    seller.element.createdDate,
                                                                    NULL AS distributionCenterZipcode,
                                                                    NULL AS distributionCenterStreet,
                                                                    NULL AS distributionCenterNumber,
                                                                    NULL AS distributionCenterNeighborhood,
                                                                    NULL AS distributionCenterCity,
                                                                    NULL AS distributionCenterState,
                                                                    CASE
                                                                        WHEN address.element.type = 'RETURN' THEN address.element.zipcode
                                                                        ELSE NULL
                                                                    END AS returnZipcode,
                                                                    CASE
                                                                        WHEN address.element.type = 'RETURN' THEN address.element.street
                                                                        ELSE NULL
                                                                    END AS returnStreet,
                                                                    CASE
                                                                        WHEN address.element.type = 'RETURN' THEN address.element.number
                                                                        ELSE NULL
                                                                    END AS returnNumber,
                                                                    CASE
                                                                        WHEN address.element.type = 'RETURN' THEN address.element.neighborhood
                                                                        ELSE NULL
                                                                    END AS returnNeighborhood,
                                                                    CASE
                                                                        WHEN address.element.type = 'RETURN' THEN address.element.city
                                                                        ELSE NULL
                                                                    END AS returnCity,
                                                                    CASE
                                                                        WHEN address.element.type = 'RETURN' THEN address.element.state
                                                                        ELSE NULL
                                                                    END AS returnState
                                                FROM netshoes_marketplace.suppliers supplier,
                                                        UNNEST(supplier.sellers.list) seller,
                                                        UNNEST(seller.element.addresses.list) address)
                                                GROUP BY registrationNumber,
                                                        companyName,
                                                        tradeName,
                                                        sellerID,
                                                        name,
                                                        active,
                                                        fastLane,
                                                        createdDate""", 'maga-bigdata')


base_bq['CNPJ']=base_bq['CNPJ'].astype(float)
dados_contatos_sellers_nets_final = pd.merge(dados_contatos_sellers_nets_v2, base_bq, how='left', on='CNPJ')
dados_contatos_sellers_nets_final['seller_id'] = dados_contatos_sellers_nets_final['seller_id'].astype(str)
dados_contatos_sellers_nets_final['seller_id'] = dados_contatos_sellers_nets_final['seller_id'].str[:5]
dados_contatos_sellers_nets_final['LOJISTA'] = dados_contatos_sellers_nets_final['LOJISTA'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.replace(' ','')

In [ ]:
# Busca de sellers que receberam pesquisa mês passado

receberam_meses_passados_nets = receberam_meses_passados.query("base == 'Netshoes'")
base_nets_v1 = dados_contatos_sellers_nets_final[~dados_contatos_sellers_nets_final['LOJISTA'].isin(receberam_meses_passados_nets['seller_id'].tolist())]
# Concatenação base pedidos + base contatos
base_nets_v2 = pd.merge(base_pedidos_final,base_nets_v1, how='inner', on='seller_id')
# Salvar dados
base_nets_v2['modalidade'] = base_nets_v2['Tipo_Envio'].map({
    'NSE': 'Netshoes (Ativo-MLE)',
}).fillna('Netshoes (Ativo)')
base_nets_v3 = base_nets_v2[base_nets_v2['active'] == True][[
    'LOJISTA_x',
    'email',
    'modalidade'
]].rename(columns={
    'LOJISTA_x': 'seller_id',
}).drop_duplicates(subset=['email'])
etl_funcs.exportar_para_planilha('base_pesquisa_nets', 'base', base_nets_v3, False)